In [3]:
%cd ..


/Users/koloss/Desktop/CS224N/final_project


In [169]:
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from transformers import DistilBertTokenizerFast
import util

In [6]:
import pandas as pd
import numpy as np

In [168]:
from data_augmentation import transform_context
from debug_german import get_dataset2

In [25]:
CHANGES = dict(zip([" game ", " was "], [" lame ", " sos "]))


In [18]:
dataset_dict = get_dataset2(datasets = 'duorc,race', data_dir ='datasets/oodomain_train' , split_name="train", debug=-1)


In [256]:
changes=CHANGES
original_df = pd.DataFrame({x: dataset_dict[x] for x in dataset_dict})
df= original_df.copy()
#df['context'] = df.context.str.strip().replace(changes, regex=True)

df['start_char'] = df.answer.apply(lambda x: x['answer_start'][0])
df['end_char'] = df['start_char'] + df.answer.apply(lambda x: len(x['text'][0]))
df['final_answer0'] = [A[B:C] for A, B, C in zip(df.context, df['start_char'], df['end_char'])]

df['context_before'] = [A[:C] for A, C in zip(df.context, df['start_char'])]
df['context_after'] = [A[C:] for A, C in zip(df.context, df['end_char'])]
df['context_answer'] = [A[B:C] for A, B, C in zip(df.context, df['start_char'], df['end_char'])] 

df['context_before']=df['context_before'].apply(lambda t: transform_context(t, p=0.1))
df['context_after']=df['context_after'].apply(lambda t: transform_context(t, p=0.1))

df['new_context'] = df['context_before']+df['context_answer'] +df['context_after']
df['new_start_char'] = df['context_before'].str.len()

df['new_end_char'] = df['new_start_char'] + df.answer.apply(lambda x: len(x['text'][0]))
df['final_answer'] = [A[B:C] for A, B, C in zip(df.new_context, df['new_start_char'], df['new_end_char'])]



#new_dataset_dict = pd.concat([original_df, df[[i for i in dataset_dict.keys()]]]).to_dict(orient='list')

In [272]:
df['new_end_char'].iloc[2]

3003

In [273]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [267]:
for i,l in enumerate(dataset_dict['answer']):
    l['answer_start'][0] = df['new_end_char'].iloc[i]

In [268]:
dataset_dict['answer']

[{'answer_start': [2562], 'text': ['acid']},
 {'answer_start': [27], 'text': ['1927']},
 {'answer_start': [3003], 'text': ['Mary-Anne']},
 {'answer_start': [3747], 'text': ['Emily']},
 {'answer_start': [2437], 'text': ['Jill']},
 {'answer_start': [3957, 3136], 'text': ['36', 'Room 36']},
 {'answer_start': [1350, 472], 'text': ['Martha', 'Liza Merril']},
 {'answer_start': [669], 'text': ['Wart']},
 {'answer_start': [994], 'text': ['Merlin']},
 {'answer_start': [223], 'text': ['from the heavens in London town']},
 {'answer_start': [995], 'text': ['Merlin']},
 {'answer_start': [3389], 'text': ['Archimedes']},
 {'answer_start': [3366], 'text': ['Archimedes']},
 {'answer_start': [2283], 'text': ['Arthur']},
 {'answer_start': [1004], 'text': ['Merlin']},
 {'answer_start': [2759, 2591], 'text': ['Pud', 'Pud']},
 {'answer_start': [2696, 2591], 'text': ['Pud', 'Pud']},
 {'answer_start': [2996, 2875], 'text': ['Granny Nellie', 'Granny Nellie']},
 {'answer_start': [1744, 1702], 'text': ['Mr. Grim

In [263]:
df.answer.apply(pd.Series)['answer_start'].str[0]

0      2476
1        26
2      2873
3      3659
4      2348
       ... 
249    1401
250     890
251    1406
252     720
253    1250
Name: answer_start, Length: 254, dtype: int64

In [237]:
df.loc[5, 'context'][3136:]

'Room 36 of the hotel and about the supernatural underworld that the hotel conceals. The hotel was built over one of the Seven Gates of Hell, and Schweik has been the Guardian. Emily is about to reveal more when her hands wander over to a canvas depicting a desolate vision of lost souls in a terrible and arid landscape. Suddenly afraid, Emily says that the painting was painted by Schweik before he died, and she runs out of the hotel parlor into the night. But Liza notices a disquieting fact about her sudden departure: Emily made no footfalls on the bare wooden boards as she ran, and neither did her seeing-eye dog.The next day, Liza ventures nervously into Room 36, a dingy phantasmal of sheet-covered furniture and shafts of dusty light. She finds an ancient book, whose weirdly flesh-like cover bears the single word'

In [255]:
a=df.answer.apply(pd.Series)
len(a.to_dict('records'))

254

In [247]:
a.answer_start.apply(lambda x :x[0])=1

SyntaxError: can't assign to function call (<ipython-input-247-49797c6c73c0>, line 1)

In [222]:
 df.answer.apply(lambda x : pd.Series(x)['answer_start'][0])

0      2476
1        26
2      2873
3      3659
4      2348
       ... 
249    1401
250     890
251    1406
252     720
253    1250
Name: answer, Length: 254, dtype: int64

In [202]:
 df.answer.apply(pd.Series)

,answer_start,text
0,[2476],[acid]
1,[26],[1927]
2,[2873],[Mary-Anne]
3,[3659],[Emily]
4,[2348],[Jill]
...,...,...
249,[1401],[happy]
250,[890],[nothing.]
251,[1406],[law enforcement]
252,[720],[white elephant]


In [182]:
df[['final_answer', 'final_answer0', 'context_answer']]

,final_answer,final_answer0,context_answer
0,acid,acid,acid
1,1927,1927,1927
2,Mary-Anne,Mary-Anne,Mary-Anne
3,Emily,Emily,Emily
4,Jill,Jill,Jill
...,...,...,...
249,happy,happy,happy
250,nothing.,nothing.,nothing.
251,law enforcement,law enforcement,law enforcement
252,white elephant,white elephant,white elephant


In [172]:
df['context_before'].iloc[0]

"New Orleans, Louisiana, 1927. An enraged posse of man descend on the isolated Seven Doors Hotel bass in the swamps. They catch an artist call Schweik( Antoine Saint John), who is cloister there. accuse him of being a warlock, Schweik is dragged down to the root_cellar where he is viciously beat with grave chains, tortured with quicklime acid, and crucified with his wrist nailed to a root_cellar wall, despite his awful warn of evil to be unleashed.New Orleans, 1981. Liza Merril( Catriona MacColl) is a young womanhood who arrives from New York City to claim the hotel as her inheritance. No sooner has architect supporter Marin Avery( Michele Mirabella) begin to show her about the place, strange incident begin to find. A cougar( Anthony Flees) waterfall off his swindle and is dreadfully injured, coughing up rake and babbling about, `` the eye, the eye. '' Dr. John McCabe( David Warbeck) arrive to take the injured homo to the hospital, and extend Liza some sympathy. Next, a plumber, named 

In [31]:
id_example = 249

In [ ]:
def is_valid(word):
    

In [111]:
question = df.loc[id_example, 'question']
answer = df.loc[id_example, 'answer']
answer_word = answer["text"][0]
context = df.loc[id_example, 'context']

In [119]:
answer_word="may"

In [148]:
text = context[-200:]

In [164]:
df


,question,context,id,answer,label,start_char,end_char,final_answer
0,What was Jill's mother's face burned by?,"\n\nNew Orleans, Louisiana, 1927. An enraged p...",d94a42693350473581ff79dc91c91e04,"{'answer_start': [2476], 'text': ['acid']}",0,2476,2480,acid
1,What year did a lynch mod muder someone who th...,"\n\nNew Orleans, Louisiana, 1927. An enraged p...",f41dbe24bed44870a8ad36c87dda59a2,"{'answer_start': [26], 'text': ['1927']}",0,26,30,1927
2,Whose corpse is in the morgue?,"\n\nNew Orleans, Louisiana, 1927. An enraged p...",34f8093a16b64c8097bdaa03cccdef37,"{'answer_start': [2873], 'text': ['Mary-Anne']}",0,2873,2882,Mary-Anne
3,What is the name of the blind girl?,"\n\nNew Orleans, Louisiana, 1927. An enraged p...",b6a13cac6289435697e8ff98d55854a9,"{'answer_start': [3659], 'text': ['Emily']}",0,3659,3664,Emily
4,Whose corpse does Liza see?,"\n\nNew Orleans, Louisiana, 1927. An enraged p...",52ffe790d24a41669c08a240c1d45114,"{'answer_start': [2348], 'text': ['Jill']}",0,2348,2352,Jill
...,...,...,...,...,...,...,...,...
249,How did the writer feel after he jumped out of...,"When we are unfamiliar with something, we may ...",32137f8962fe4101a8efb30e742c8bb5,"{'answer_start': [1401], 'text': ['happy']}",1,1401,1406,happy
250,Recorded history has taught us,In a few states where racial prejudice is seri...,3372b6a3133549f4af5df9b725757e8e,"{'answer_start': [890], 'text': ['nothing.']}",1,890,898,nothing.
251,According the author the best way to solve rac...,In a few states where racial prejudice is seri...,b65b933a7b15401790b44f243f3e3e96,"{'answer_start': [1406], 'text': ['law enforce...",1,1406,1421,law enforcement
252,Which expression has an historical background?,In many cultures white is often associated wit...,cc7e1bf548c242af8937d7c2b6851563,"{'answer_start': [720], 'text': ['white elepha...",1,720,734,white elephant


In [159]:
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from random import randint
import nltk.data

# Load a text file if required
output = ""
counter = 0

# Load the pretrained neural net
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Tokenize the text
tokenized = tokenizer.tokenize(text)

# Get the list of words from the entire text
words = word_tokenize(text)

# Identify the parts of speech
tagged = nltk.pos_tag(words)

for i in range(0,len(words)):
    if words[i]==answer_word:
        output = output + " " + words[i]
        continue
    replacements = []
   
    # Only replace nouns with nouns, vowels with vowels etc.
    for syn in wordnet.synsets(words[i]):
        
        # Do not attempt to replace proper nouns or determiners
        if tagged[i][1] == 'NNP' or tagged[i][1] == 'DT':
            break
        
        # The tokenizer returns strings like NNP, VBP etc
        # but the wordnet synonyms has tags like .n.
        # So we extract the first character from NNP ie n
        # then we check if the dictionary word has a .n. or not 
        word_type = tagged[i][1][0].lower()
        if syn.name().find("."+word_type+"."):
            # extract the word only
            
            r = syn.name()[0:syn.name().find(".")]
            if r != words[i]:
                replacements.append(r)
            

    if len(replacements) > 0:
        # Choose a random replacement
        
        replacement = replacements[randint(0,len(replacements)-1)]
        print(words[i] + " replaced by " +replacement)
        counter+=  len(replacement)-len(words[i])
        output = output + " " + replacement
        
    else:
        # If no replacement could be found, then just use the
        # original word
        if words[i] in string.punctuation:
            output = output  + words[i]
        else:
            
            output = output + " " + words[i]
print("BEFORE : " + text)
print("AFTER : " + output)

felt replaced by feel
quick replaced by flying
pull replaced by perpetrate
big replaced by adult
parachute replaced by chute
had replaced by give_birth
opened replaced by open
It replaced by information_technology
was replaced by be
like replaced by comparable
best replaced by good
surprise replaced by storm
I replaced by i
ever replaced by always
had replaced by accept
I replaced by iodine
looked replaced by look
down replaced by polish
There replaced by there
were replaced by be
rivers replaced by river
trees replaced by tree
fields replaced by airfield
houses replaced by house
I replaced by one
heard replaced by hear
soft replaced by gentle
sound replaced by heavy
air replaced by air_travel
was replaced by cost
fun replaced by playfulness
BEFORE : felt a quick pull. My big parachute had opened! It was like the best surprise I'd ever had. I looked down. There were rivers, trees, fields and houses. I heard the soft sound of the air. This was fun!
AFTER :  feel a flying perpetrate. My 

In [158]:
import string 
")" in string.punctuation 

True

In [137]:
counter

2

In [98]:
print(text)

When we are unfamiliar with something, we may feel nervous and fearful


In [163]:
get_synonyms("I")

['',
 'ace',
 'i',
 'iodine',
 'one',
 'single',
 'ane',
 'iodin',
 'atomic number ',
 'unity']

In [75]:
print(text)

When we are unfamiliar with something, we may feel nervous and fearful


In [70]:
print(output)

 When we be unfamiliar with something , we may feel skittish and fearful


In [67]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/koloss/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [52]:
chars = get_only_chars(context)
new_chars = synonym_replacement(chars.split(" "), n=100)

In [55]:
len(chars)

1545

In [56]:
len(new_chars)

1784

In [46]:
for word in context.split(" ")[10:20]:
    #if is_valid(word)
    print(word , get_synonyms(word))

and []
fearful. []
The []
help ['supporter', 'assist', 'help oneself', 'serve', 'assistant', 'helper', 'assistance', 'service', 'aid', 'facilitate', 'avail']
of []
others []
is ['be', 'exist', 'embody', 'follow', 'represent', 'personify', 'cost', 'make up', 'live', 'constitute', 'comprise', 'equal']
a ['type a', 'group a', 'antiophthalmic factor', 'vitamin a', 'angstrom', 'axerophthol', 'deoxyadenosine monophosphate', 'amp', 'angstrom unit', 'ampere', 'adenine']
good ['in force', 'well', 'beneficial', 'honorable', 'in effect', 'serious', 'upright', 'dependable', 'skilful', 'soundly', 'unspoilt', 'ripe', 'trade good', 'estimable', 'proficient', 'near', 'goodness', 'commodity', 'sound', 'full', 'practiced', 'expert', 'adept', 'thoroughly', 'honest', 'unspoiled', 'skillful', 'safe', 'effective', 'undecomposed', 'salutary', 'respectable', 'dear', 'right', 'secure', 'just']
way ['agency', 'room', 'direction', 'right smart', 'elbow room', 'path', 'way of life', 'style', 'manner', 'fashion', 

In [45]:
get_synonyms('unfamiliar')

[]

In [38]:
answer

{'answer_start': [1401], 'text': ['happy']}

In [32]:
df.loc[id_example, 'context']

'When we are unfamiliar with something, we may feel nervous and fearful. The help of others is a good way to make us pull through. I write for a big newspaper, and I wanted to write a story about parachute jumping. To make it a realistic as well as exciting story, I decided that I first had to make a jump myself. Unfortunately, I am not good at any sport of any kind, let alone parachute jumping. My friend Mr. Smith was willing to give me a hand. He took me to a ground school. The first day\'s training included several hours of instruction but not my first drop from an airplane. For this, I had to wait until the following morning. The next morning, after a hurried, anxious breakfast, I was taken to the airfield. There, two heavy parachutes were put on my back. Then I was asked to make my way to a small plane which had just stopped slowly on the runway. Once on the plane, it was soon climbing. I began to feel nervous. As we reached one thousand meters, Harry, my teacher, hooked a line fr

In [24]:
df

,question,context,id,answer,label,start_char,end_char,final_answer
0,What was Jill's mother's face burned by?,"\n\nNew Orleans, Louisiana, 1927. An enraged p...",d94a42693350473581ff79dc91c91e04,"{'answer_start': [2476], 'text': ['acid']}",0,2476,2480,acid
1,What year did a lynch mod muder someone who th...,"\n\nNew Orleans, Louisiana, 1927. An enraged p...",f41dbe24bed44870a8ad36c87dda59a2,"{'answer_start': [26], 'text': ['1927']}",0,26,30,1927
2,Whose corpse is in the morgue?,"\n\nNew Orleans, Louisiana, 1927. An enraged p...",34f8093a16b64c8097bdaa03cccdef37,"{'answer_start': [2873], 'text': ['Mary-Anne']}",0,2873,2882,Mary-Anne
3,What is the name of the blind girl?,"\n\nNew Orleans, Louisiana, 1927. An enraged p...",b6a13cac6289435697e8ff98d55854a9,"{'answer_start': [3659], 'text': ['Emily']}",0,3659,3664,Emily
4,Whose corpse does Liza see?,"\n\nNew Orleans, Louisiana, 1927. An enraged p...",52ffe790d24a41669c08a240c1d45114,"{'answer_start': [2348], 'text': ['Jill']}",0,2348,2352,Jill
...,...,...,...,...,...,...,...,...
249,How did the writer feel after he jumped out of...,"When we are unfamiliar with something, we may ...",32137f8962fe4101a8efb30e742c8bb5,"{'answer_start': [1401], 'text': ['happy']}",1,1401,1406,happy
250,Recorded history has taught us,In a few states where racial prejudice is seri...,3372b6a3133549f4af5df9b725757e8e,"{'answer_start': [890], 'text': ['nothing.']}",1,890,898,nothing.
251,According the author the best way to solve rac...,In a few states where racial prejudice is seri...,b65b933a7b15401790b44f243f3e3e96,"{'answer_start': [1406], 'text': ['law enforce...",1,1406,1421,law enforcement
252,Which expression has an historical background?,In many cultures white is often associated wit...,cc7e1bf548c242af8937d7c2b6851563,"{'answer_start': [720], 'text': ['white elepha...",1,720,734,white elephant


In [7]:

def get_dataset(args, datasets, data_dir, tokenizer, split_name, debug=-1):
    datasets = datasets.split(',')
    dataset_dict = None
    dataset_name = ''
    label = 0
    for dataset in datasets:
        dataset_name += f'_{dataset}'
        dataset_dict_curr = util.read_squad(f'{data_dir}/{dataset}')
        dataset_dict_curr['label'] = label
        dataset_dict = util.merge(dataset_dict, dataset_dict_curr)
        label += 1
    data_encodings = read_and_process(args, tokenizer, dataset_dict, data_dir, dataset_name, split_name)
    return util.QADataset(data_encodings, train=(split_name == 'train')), dataset_dict, label


In [76]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
datasets = 'duorc,race,relation_extraction'
data_dir = 'datasets/oodomain_train'

datasets = datasets.split(',')
dataset_dict = None
dataset_name = ''
label = 0
for dataset in datasets:
    dataset_name += f'_{dataset}'
    dataset_dict_curr = util.read_squad(f'{data_dir}/{dataset}')
    dataset_dict_curr['label'] = label
    dataset_dict = util.merge(dataset_dict, dataset_dict_curr)
    label += 1
#data_encodings = read_and_process(args, tokenizer, dataset_dict, data_dir, dataset_name, split_name)
df = pd.DataFrame({x: dataset_dict[x] for x in dataset_dict if x not in ['label']})

df['start_char'] = df.answer.apply(lambda x : x['answer_start'][0])
df['end_char'] = df['start_char']+ df.answer.apply(lambda x : len(x['text'][0]))

df['final_answer'] = [A[B:C] for A, B, C in zip(df.context, df['start_char'],df['end_char'])]



In [227]:
ind = 34
proportion_random = 0.05


context = dataset_dict['context'][ind]
answer = dataset_dict['answer'][ind]
question = dataset_dict['question'][ind]

all_words = np.unique(get_only_chars(context).split(' ')).tolist()
size_random = int(len(all_words)*proportion_random)


random.sample(all_words,size_random)



['underground',
 'hotel',
 's',
 'london',
 'dead',
 'use',
 'with',
 'orders',
 'some',
 'blusters',
 'cia',
 'spy',
 'prototype',
 'several',
 'lt',
 'disguised',
 'customs',
 'girl',
 'new']

In [226]:
context.split(" ")



['\n\nA',
 'Huey',
 'helicopter',
 'flies',
 'over',
 'the',
 'Alaskan',
 'wilderness,',
 'its',
 'pilots',
 'looking',
 'for',
 'someone',
 'below.',
 'That',
 'someone,',
 'Major',
 'Mitchell',
 'Gant',
 'USAF',
 '(Rtd)',
 '(Clint',
 'Eastwood),',
 'hears',
 'the',
 'helicopter',
 'approaching',
 'and',
 'instantly',
 'breaks',
 'into',
 'a',
 'dead',
 'run',
 'back',
 'toward',
 'his',
 'cabin,',
 'where',
 'he',
 'takes',
 'a',
 'shotgun',
 'off',
 'its',
 'rack',
 'and',
 'cocks',
 'it.',
 'As',
 'the',
 'helicopter',
 'lands,',
 'Gant',
 'lapses',
 'into',
 'a',
 'post-tramautic',
 'memory',
 'of',
 'a',
 'nightmare',
 'that',
 'he',
 'lived',
 'through',
 'in',
 'Vietnam:',
 'shot',
 'down',
 'over',
 'the',
 'North',
 'in',
 'his',
 'A-4,',
 'he',
 'was',
 'being',
 'taken',
 'to',
 'a',
 'prison',
 'camp',
 'when',
 'two',
 'Hueys',
 'machine-gunned',
 'his',
 'captors.',
 'Gant',
 'suffered',
 'personal',
 'trauma',
 'when',
 'an',
 'overflying',
 'A-4',
 'dropped',
 'an',
 '

In [201]:
size_random

803

In [189]:
get_only_chars(context).split(' ')

['a',
 'huey',
 'helicopter',
 'flies',
 'over',
 'the',
 'alaskan',
 'wilderness',
 'its',
 'pilots',
 'looking',
 'for',
 'someone',
 'below',
 'that',
 'someone',
 'major',
 'mitchell',
 'gant',
 'usaf',
 'rtd',
 'clint',
 'eastwood',
 'hears',
 'the',
 'helicopter',
 'approaching',
 'and',
 'instantly',
 'breaks',
 'into',
 'a',
 'dead',
 'run',
 'back',
 'toward',
 'his',
 'cabin',
 'where',
 'he',
 'takes',
 'a',
 'shotgun',
 'off',
 'its',
 'rack',
 'and',
 'cocks',
 'it',
 'as',
 'the',
 'helicopter',
 'lands',
 'gant',
 'lapses',
 'into',
 'a',
 'post',
 'tramautic',
 'memory',
 'of',
 'a',
 'nightmare',
 'that',
 'he',
 'lived',
 'through',
 'in',
 'vietnam',
 'shot',
 'down',
 'over',
 'the',
 'north',
 'in',
 'his',
 'a',
 'he',
 'was',
 'being',
 'taken',
 'to',
 'a',
 'prison',
 'camp',
 'when',
 'two',
 'hueys',
 'machine',
 'gunned',
 'his',
 'captors',
 'gant',
 'suffered',
 'personal',
 'trauma',
 'when',
 'an',
 'overflying',
 'a',
 'dropped',
 'an',
 'incendiary',
 

In [196]:
all_words.tolist()

['',
 'a',
 'able',
 'about',
 'advantage',
 'aerial',
 'affording',
 'after',
 'agree',
 'aim',
 'air',
 'airport',
 'alaskan',
 'all',
 'allow',
 'allowed',
 'also',
 'an',
 'and',
 'any',
 'anything',
 'apologizes',
 'approaching',
 'are',
 'around',
 'arrest',
 'arthur',
 'as',
 'at',
 'attache',
 'aubrey',
 'back',
 'base',
 'battle',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'bilyarsk',
 'blackmailed',
 'blusters',
 'both',
 'breaks',
 'bridge',
 'briefing',
 'briefings',
 'british',
 'buckholz',
 'businessman',
 'but',
 'button',
 'by',
 'cabin',
 'camp',
 'capabilities',
 'captain',
 'captors',
 'ceiling',
 'center',
 'cheap',
 'cia',
 'clint',
 'cocks',
 'codename',
 'col',
 'colley',
 'colonel',
 'combat',
 'command',
 'compromises',
 'concerning',
 'conducted',
 'conversation',
 'corrupt',
 'cotton',
 'crack',
 'customs',
 'cuts',
 'david',
 'dawn',
 'day',
 'dead',
 'defense',
 'delivering',
 'descision',
 'development',
 'device',
 'disguised'

In [198]:
random.sample(all_words,10)


['members',
 'lived',
 'plans',
 'wilderness',
 'allowed',
 'outside',
 'approaching',
 'if',
 'vietnam',
 'stepping']

In [165]:
get_synonyms(word)

['senior pilot',
 'police chief',
 'police captain',
 'maitre d',
 'chieftain',
 'sea captain',
 'headwaiter',
 'maitre dhotel',
 'master',
 'skipper',
 'captain']

In [169]:
'word' in 'qwertyuiopasdfghjklzxcvbnm '

False

In [ ]:
	new_words = words.copy()
	random_word_list = list(set([word for word in words if (word not in stop_words) and (word[0].islower()) ]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: #only replace up to n words
			break

In [85]:
answer

{'answer_start': [2875, 2875], 'text': ['Granny Nellie', 'Granny Nellie']}

In [73]:
dataset_dict['answer']

[{'answer_start': [2476], 'text': ['acid']},
 {'answer_start': [26], 'text': ['1927']},
 {'answer_start': [2873], 'text': ['Mary-Anne']},
 {'answer_start': [3659], 'text': ['Emily']},
 {'answer_start': [2348], 'text': ['Jill']},
 {'answer_start': [3804, 3136], 'text': ['36', 'Room 36']},
 {'answer_start': [1307, 472], 'text': ['Martha', 'Liza Merril']},
 {'answer_start': [658], 'text': ['Wart']},
 {'answer_start': [959], 'text': ['Merlin']},
 {'answer_start': [201], 'text': ['from the heavens in London town']},
 {'answer_start': [959], 'text': ['Merlin']},
 {'answer_start': [3265], 'text': ['Archimedes']},
 {'answer_start': [3265], 'text': ['Archimedes']},
 {'answer_start': [2236], 'text': ['Arthur']},
 {'answer_start': [959], 'text': ['Merlin']},
 {'answer_start': [2591, 2591], 'text': ['Pud', 'Pud']},
 {'answer_start': [2591, 2591], 'text': ['Pud', 'Pud']},
 {'answer_start': [2875, 2875], 'text': ['Granny Nellie', 'Granny Nellie']},
 {'answer_start': [1702, 1702], 'text': ['Mr. Grime

In [65]:
get_only_chars(dataset_dict['context'][34].split('\n'))

AttributeError: 'list' object has no attribute 'replace'

In [69]:
context_line = 
get_only_chars(df['context'][34]).split(" ")

['a',
 'huey',
 'helicopter',
 'flies',
 'over',
 'the',
 'alaskan',
 'wilderness',
 'its',
 'pilots',
 'looking',
 'for',
 'someone',
 'below',
 'that',
 'someone',
 'major',
 'mitchell',
 'gant',
 'usaf',
 'rtd',
 'clint',
 'eastwood',
 'hears',
 'the',
 'helicopter',
 'approaching',
 'and',
 'instantly',
 'breaks',
 'into',
 'a',
 'dead',
 'run',
 'back',
 'toward',
 'his',
 'cabin',
 'where',
 'he',
 'takes',
 'a',
 'shotgun',
 'off',
 'its',
 'rack',
 'and',
 'cocks',
 'it',
 'as',
 'the',
 'helicopter',
 'lands',
 'gant',
 'lapses',
 'into',
 'a',
 'post',
 'tramautic',
 'memory',
 'of',
 'a',
 'nightmare',
 'that',
 'he',
 'lived',
 'through',
 'in',
 'vietnam',
 'shot',
 'down',
 'over',
 'the',
 'north',
 'in',
 'his',
 'a',
 'he',
 'was',
 'being',
 'taken',
 'to',
 'a',
 'prison',
 'camp',
 'when',
 'two',
 'hueys',
 'machine',
 'gunned',
 'his',
 'captors',
 'gant',
 'suffered',
 'personal',
 'trauma',
 'when',
 'an',
 'overflying',
 'a',
 'dropped',
 'an',
 'incendiary',
 

In [45]:
len(np.unique(df['context']))

245

In [46]:
len(df)

381

In [31]:
df['context'].str.split("\n")[20]

["Brink (Sir Cedric Hardwicke) has recently taken Pud's (Bobs Watson) parents in an auto wreck. Brink later comes for Gramps (Lionel Barrymore). Believing Brink to be an ordinary stranger, the crotchety old Gramps orders Mr. Brink off the property. Pud comes out of the house and asks who the stranger was. Gramps is surprised and relieved that someone else could see the stranger; he was not merely a dream or apparition.",
 'Pud tells Gramps that when he does a good deed, he will be able to make a wish. Because his apples are constantly being stolen, Gramps wishes that anyone who climbs up his apple tree will have to stay there until he permits them to climb down. Pud inadvertently tests the wish when he has trouble coming down from the tree himself, becoming free only when Gramps says he can.',
 "Pud's busybody Aunt Demetria (Eily Malyon) has designs on Pud and the money left him by his parents. Gramps spends much time fending off her efforts to adopt the boy.",
 'Brink takes Granny Nel

In [23]:
np.unique(np.concatenate(df['context'].str.split(" ")))[:100]

array(['', '\n\nA', '\n\nIn', '\n\nIts', '\n\nNew', '\n\nThe', '!', '"',
       '".It', '"1\'vc', '"12\'\').', '"20,000"', '"2E".', '"5:11', '"A',
       '"After', '"All', '"Amir!', '"And', '"As', '"But', '"Come',
       '"Danish', '"Do', '"Father!', '"Father!"', '"Firefox"',
       '"Friends,', '"Get', '"Good', '"How', '"I', '"I\'m', '"I\'ve',
       '"If', '"In', '"Is', '"It', '"It\'s', '"Ivy', '"Jump!"', '"Kids',
       '"Like', '"Many', '"Mastermind"', '"Mastermind",', '"Measures',
       '"MiG"', '"Money"', '"My', '"No', '"No,', '"Oh,', '"Oh?', '"Our',
       '"Pepsi', '"Please', '"Possible,', '"Pygmalion",', '"Queensland',
       '"Remove', '"Rolling', '"Seeing', '"Severe', '"Skeptics', '"So',
       '"Some', '"Stop!"', '"SurviVR', '"Talk,', '"The', '"There',
       '"These', '"They', '"They\'re', '"This', '"Treat', '"Uncle"',
       '"Waiting', '"We', '"We\'re', '"When', '"While', '"Whitewash"',
       '"Whoso', '"Why', '"Will', '"With', '"Would', '"You', '"You\'d',
       '"You

In [21]:
df.loc[0, "context"].split(' ')

['\n\nNew',
 'Orleans,',
 'Louisiana,',
 '1927.',
 'An',
 'enraged',
 'posse',
 'of',
 'men',
 'descend',
 'on',
 'the',
 'isolated',
 'Seven',
 'Doors',
 'Hotel',
 'deep',
 'in',
 'the',
 'swamps.',
 'They',
 'grab',
 'an',
 'artist',
 'called',
 'Schweik',
 '(Antoine',
 'Saint',
 'John),',
 'who',
 'is',
 'cloistered',
 'there.',
 'Accusing',
 'him',
 'of',
 'being',
 'a',
 'warlock,',
 'Schweik',
 'is',
 'dragged',
 'down',
 'to',
 'the',
 'cellar',
 'where',
 'he',
 'is',
 'savagely',
 'beaten',
 'with',
 'heavy',
 'chains,',
 'tortured',
 'with',
 'quicklime',
 'acid,',
 'and',
 'crucified',
 'with',
 'his',
 'wrists',
 'nailed',
 'to',
 'a',
 'cellar',
 'wall,',
 'despite',
 'his',
 'dire',
 'warnings',
 'of',
 'evil',
 'to',
 'be',
 'unleashed.New',
 'Orleans,',
 '1981.',
 'Liza',
 'Merril',
 '(Catriona',
 'MacColl)',
 'is',
 'a',
 'young',
 'woman',
 'who',
 'arrives',
 'from',
 'New',
 'York',
 'City',
 'to',
 'claim',
 'the',
 'hotel',
 'as',
 'her',
 'inheritance.',
 'No',
 

In [34]:
df = pd.DataFrame({x: dataset_dict[x] for x in dataset_dict if x not in ['label']})
df['start_char'] = df.answer.apply(lambda x : x['answer_start'][0])
df['end_char'] = df['start_char']+ df.answer.apply(lambda x : len(x['text'][0]))
df['final_answer'] = [A[B:C] for A, B, C in zip(df.context, df['start_char'],df['end_char'])]



In [35]:
line = df.loc[20, "context"].split('\n')[0]
#clean_line = get_only_chars(line).split(" ")
clean_line = line.split(" ")
modif_line = synonym_replacement(clean_line, 20)
modif_line


"verge (Sir Cedric Hardwicke) has of late have Pud's (Bobs Watson) raise in an machine wreck. verge tardy fare for Gramps (Lionel Barrymore). consider verge to be an average stranger, the cantankerous previous Gramps dictate mister verge off the property. pud fare out of the house and inquire who the alien was. Gramps is surprised and sticking that individual else could pick up the stranger; he was not merely a ambition or apparition."

In [1]:
line

NameError: name 'line' is not defined

In [40]:
clean_line

'brink sir cedric hardwicke has recently taken puds bobs watson parents in an auto wreck brink later comes for gramps lionel barrymore believing brink to be an ordinary stranger the crotchety old gramps orders mr brink off the property pud comes out of the house and asks who the stranger was gramps is surprised and relieved that someone else could see the stranger he was not merely a dream or apparition '

In [27]:
words=df.loc[0,'context'].split(" ")
new_word = synonym_replacement(words, 100)


In [29]:
len(new_word)


703

In [30]:
len(words)

673

In [169]:
changes=dict(zip([" game ", " set "], [" lame ", " bet "]))

df = pd.DataFrame({x: dataset_dict[x] for x in dataset_dict if x not in ['label']})
df['start_char'] = df.answer.apply(lambda x : x['answer_start'][0])
df['end_char'] = df['start_char']+ df.answer.apply(lambda x : len(x['text'][0]))
df['final_answer'] = [A[B:C] for A, B, C in zip(df.context, df['start_char'],df['end_char'])]
df['context'] = df.context.str.strip().replace(changes,regex=True)

##df['new_context'] = df.context.str.strip().replace(changes,regex=True)
##df['new_answer'] = [A[B:C] for A, B, C in zip(df['new_context'], df['start_char'],df['end_char'])]
new_dataset_dict = df[[i for i in dataset_dict.keys() if i!= 'label']].to_dict()
new_dataset_dict ['label'] = dataset_dict['label']

In [156]:
#df[[i for i in dataset_dict.keys() if i!= 'label']].to_dict()

In [172]:
df.head()

,question,context,id,answer,start_char,end_char,final_answer
0,What was Jill's mother's face burned by?,"New Orleans, Louisiana, 1927. An enraged posse...",d94a42693350473581ff79dc91c91e04,"{'answer_start': [2476], 'text': ['acid']}",2476,2480,acid
1,What year did a lynch mod muder someone who th...,"New Orleans, Louisiana, 1927. An enraged posse...",f41dbe24bed44870a8ad36c87dda59a2,"{'answer_start': [26], 'text': ['1927']}",26,30,1927
2,Whose corpse is in the morgue?,"New Orleans, Louisiana, 1927. An enraged posse...",34f8093a16b64c8097bdaa03cccdef37,"{'answer_start': [2873], 'text': ['Mary-Anne']}",2873,2882,Mary-Anne
3,What is the name of the blind girl?,"New Orleans, Louisiana, 1927. An enraged posse...",b6a13cac6289435697e8ff98d55854a9,"{'answer_start': [3659], 'text': ['Emily']}",3659,3664,Emily
4,Whose corpse does Liza see?,"New Orleans, Louisiana, 1927. An enraged posse...",52ffe790d24a41669c08a240c1d45114,"{'answer_start': [2348], 'text': ['Jill']}",2348,2352,Jill


## SR

In [173]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet 

[nltk_data] Downloading package wordnet to /Users/koloss/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /Users/koloss/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [193]:

import random
from random import shuffle

In [194]:
#stop words list
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
			'ours', 'ourselves', 'you', 'your', 'yours', 
			'yourself', 'yourselves', 'he', 'him', 'his', 
			'himself', 'she', 'her', 'hers', 'herself', 
			'it', 'its', 'itself', 'they', 'them', 'their', 
			'theirs', 'themselves', 'what', 'which', 'who', 
			'whom', 'this', 'that', 'these', 'those', 'am', 
			'is', 'are', 'was', 'were', 'be', 'been', 'being', 
			'have', 'has', 'had', 'having', 'do', 'does', 'did',
			'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
			'because', 'as', 'until', 'while', 'of', 'at', 
			'by', 'for', 'with', 'about', 'against', 'between',
			'into', 'through', 'during', 'before', 'after', 
			'above', 'below', 'to', 'from', 'up', 'down', 'in',
			'out', 'on', 'off', 'over', 'under', 'again', 
			'further', 'then', 'once', 'here', 'there', 'when', 
			'where', 'why', 'how', 'all', 'any', 'both', 'each', 
			'few', 'more', 'most', 'other', 'some', 'such', 'no', 
			'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
			'very', 's', 't', 'can', 'will', 'just', 'don', 
			'should', 'now', '']

#cleaning up text
import re
def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line


In [195]:

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words if word not in stop_words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: #only replace up to n words
			break

	#this is stupid but we need it, trust me
	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words

def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word): 
		for l in syn.lemmas(): 
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
			synonyms.add(synonym) 
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)

def eda(sentence, alpha_sr=0.1, alpha_ri=0, alpha_rs=0, p_rd=0, num_aug=9):
	
	sentence = get_only_chars(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not '']
	num_words = len(words)
	
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4)+1

	#sr
	if (alpha_sr > 0):
		n_sr = max(1, int(alpha_sr*num_words))
		for _ in range(num_new_per_technique):
			a_words = synonym_replacement(words, n_sr)
			augmented_sentences.append(' '.join(a_words))

	#ri
	if (alpha_ri > 0):
		n_ri = max(1, int(alpha_ri*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_insertion(words, n_ri)
			augmented_sentences.append(' '.join(a_words))

	#rs
	if (alpha_rs > 0):
		n_rs = max(1, int(alpha_rs*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_swap(words, n_rs)
			augmented_sentences.append(' '.join(a_words))

	#rd
	if (p_rd > 0):
		for _ in range(num_new_per_technique):
			a_words = random_deletion(words, p_rd)
			augmented_sentences.append(' '.join(a_words))

	augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
	shuffle(augmented_sentences)

	#trim so that we have the desired number of augmented sentences
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	#append the original sentence
	augmented_sentences.append(sentence)

	return augmented_sentences

In [196]:
tst = df.loc[0,'question']
tst

"What was Jill's mother's face burned by?"

In [197]:
eda(tst)

['what was jills mothers face sting by',
 'what was jills mothers face glow by',
 'what was jills mothers face up burned by',
 'what was jills mothers face burned by ']